<a href="https://colab.research.google.com/github/MustafizSaadi/DMLab/blob/main/K_medoids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import gzip
from io import StringIO
import re
import sys
import math
import statistics 
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy import stats

In [ ]:
# cd /content/drive/My\ Drive

In [ ]:
def load_dataset(file_name, threshold):
  attribute_dictionary = dict()
  max_dictionary = dict()
  min_dictionary = dict()
  category_values = dict()
  df = pd.read_csv(file_name, header = None)

  df = df.astype("float32")
  print(df.dtypes)

  full_table = df.to_numpy()
  print(full_table)

  for i in range(full_table.shape[1]):
    print(full_table[:,i])
    elements = np.unique(full_table[:,i])
    ratio =  len(elements)/full_table.shape[0]

    if ratio >= threshold:
      attribute_dictionary[i] = "numeric"
      max_dictionary[i] = max(elements)
      min_dictionary[i] = min(elements)
    else:
      attribute_dictionary[i] = "category"
      category_values[i] = elements
  
  print(attribute_dictionary)
  
  return attribute_dictionary, max_dictionary, min_dictionary, category_values, full_table

In [ ]:
def replace_nan_with_most_frequent_category(df, column):
  most_frequent_category = df.iloc[:,column].mode()[0]
  print(most_frequent_category)
  df.iloc[:,column].fillna(most_frequent_category, inplace = True)
  print(df.iloc[:,column])
  print("within replace")
  return df

In [ ]:
def load_dataset_sponge(file_name, threshold):
  attribute_dictionary = dict()
  max_dictionary = dict()
  min_dictionary = dict()
  category_values = dict()
  df = pd.read_csv(file_name, header = None)
  
  df = df.drop(df.columns[0], axis = 1)
  df = df.replace('?', np.nan)
  df = df.astype("object")
  print("dataframe type", df.dtypes)
  print("start")
  
  for i in range(df.shape[1]):
    if df.iloc[:,i].isnull().values.any():
      print(1)
      df = replace_nan_with_most_frequent_category(df, i)
    # print(df.columns[i])

  print(df.shape)
  full_table = df.to_numpy()
  print(full_table.shape[1])

  for i in range(full_table.shape[1]):
    print(full_table[:,i])
    elements = np.unique(full_table[:,i])
    ratio =  len(elements)/full_table.shape[0]

    if ratio >= threshold:
      attribute_dictionary[i] = "numeric"
      max_dictionary[i] = max(elements)
      min_dictionary[i] = min(elements)
    else:
      attribute_dictionary[i] = "category"
      category_values[i] = elements
  
  print(attribute_dictionary)
  
  return attribute_dictionary,  max_dictionary, min_dictionary, category_values, full_table

In [ ]:
def distance(attribute_dictionary,  max_dictionary, min_dictionary, data, mean):
  dist = 0
  col = 0
  for i,j in zip(data, mean):
    if attribute_dictionary[col] == "numeric":
      dist += abs(float(i)-float(j))/(max_dictionary[col] - min_dictionary[col])
    elif attribute_dictionary[col] == "category":
      if i != j:
        dist += 1
    col += 1
  # print("distance", dist/col)
  return dist/col

In [ ]:
def calculate_pairwise_distance(sillhoutte_dataset, attribute_dictionary, max_dictionary, min_dictionary):
  row = sillhoutte_dataset.shape[0]
  pairwise_distance = np.zeros((row,row))

  for i in range(row):
    for j in range(i+1, row):
      pairwise_distance[i][j] = distance(attribute_dictionary, max_dictionary, min_dictionary, sillhoutte_dataset[i], sillhoutte_dataset[j])
      pairwise_distance[j][i] = pairwise_distance[i][j]

  return pairwise_distance
  

In [ ]:
def find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, means):
  idx = -1
  min = sys.maxsize

  for mean_idx in means:
    # mean = means[i]
    dist = distance(attribute_dictionary,  max_dictionary, min_dictionary, data, database[mean_idx,:])

    if dist < min:
      min = dist
      idx = mean_idx

  return min, idx

In [ ]:
def Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means):
  clusters = dict()

  while True:
    cur_cost = 0
    flag = False

    print("means", means)

    for i in range(len(means)):
      clusters[i] = []
  
    for data in database:
  
      min, idx = find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, means)

      cur_cost += min
      clusters[means.index(idx)].append(data)

    for i in range(len(means)):
      new_means = []
      checked = []
      for mean_idx in means:
        new_means.append(mean_idx)
        checked.append(mean_idx)

      while True:
        if len(checked) == database.shape[0]:
          break

        non_representative_point = random.randint(0, database.shape[0]-1)

        if non_representative_point not in checked:
          checked.append(non_representative_point)

          new_means[i] = non_representative_point

          new_cost = 0

          for data in database:
        
            min, _ = find_closest_median(attribute_dictionary,  max_dictionary, min_dictionary, data, database, new_means)

            new_cost += min
          
          if new_cost < cur_cost:
            cur_cost = new_cost
            flag = True
            means[i] = new_means[i]
            print(means)
            break

    if not flag:
      break

  return cur_cost, means, clusters
          


In [ ]:
threshold = 0.15
# attribute_dictionary,  max_dictionary, min_dictionary, category_values, database = load_dataset('unlabeled data', threshold)
# attribute_dictionary,  max_dictionary, min_dictionary, category_values, database = load_dataset_sponge('sponge.csv', threshold)
attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('QCM3.csv', threshold)
# attribute_dictionary, max_dictionary, min_dictionary, category_values, database = load_dataset('buddymove_holidayiq.csv', threshold)
# print(category_values)
# max_k = np.linspace(1, 10, num = 10)
# # np.linspace(1, 10, num = 10)
# X = []
# Y = []
# # np.zeros((k, database.shape[1]))
iterations = 3
# for k in max_k:
#   print("******* ", k)
#   best_means = []
#   best_err = sys.maxsize
#   best_cluster = dict()
#   # min_err = sys.maxsize
#   for it in range(iterations):
#     means = []
#     err = 0
#     clusters = dict()
#     np.random.shuffle(database)
#     for i in range(int(k)):
#       # temp = []
#       # for j in range(database.shape[1]):
#       #   temp.append(database[i][j])
#       means.append(i)
#     err, means, clusters = Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means)
#     if err < best_err:
#       best_means = means
#       best_err = err
#       best_cluster = clusters
    
#   print("means", best_means)
#   print(k, best_err)
#   X.append(k)
#   Y.append(best_err)
# # means = np.array(means)

# plt.plot(X, Y)
# plt.show()

0    float32
1    float32
2    float32
3    float32
4    float32
5    float32
6    float32
7    float32
8    float32
9    float32
dtype: object
[[ -10.06  -10.62  -14.43  -18.31  -24.64  -30.56  -38.62  -45.59  -54.89
   -62.28]
 [  -9.69  -10.86  -16.73  -21.75  -28.47  -35.83  -43.65  -52.43  -61.92
   -71.27]
 [ -12.07  -14.28  -21.54  -27.92  -35.19  -43.94  -52.04  -62.49  -71.97
   -83.1 ]
 [ -14.21  -17.41  -25.91  -33.36  -41.29  -51.27  -59.94  -71.55  -81.51
   -93.83]
 [ -16.57  -20.35  -29.97  -37.84  -47.03  -57.29  -67.13  -78.96  -90.01
  -102.65]
 [ -75.61  -64.1  -122.08 -102.17 -174.79 -145.5  -214.69 -177.3  -250.83
  -207.99]
 [ -76.28  -64.53 -123.4  -103.06 -174.74 -144.98 -206.75 -170.68 -240.14
  -200.05]
 [ -76.96  -65.37 -124.5  -103.87 -169.93 -140.56 -200.89 -165.88 -232.82
  -194.7 ]
 [ -78.29  -66.49 -125.78 -104.77 -165.42 -136.17 -196.1  -161.99 -226.89
  -189.57]
 [ -79.62  -66.85 -126.61 -104.38 -162.73 -133.26 -194.87 -160.68 -221.33
  -183.81]
 [ -88

In [ ]:
best_means = []
best_err = sys.maxsize
best_cluster = dict()
# min_err = sys.maxsize
k = 3
err = 0
for it in range(iterations):
  means = []
  # err = 0
  clusters = dict()
  np.random.shuffle(database)
  for i in range(k):
    # temp = []
    # for j in range(database.shape[1]):
    #   temp.append(database[i][j])
    means.append(i)
  temp_err, means, clusters = Kmedoid(attribute_dictionary,  max_dictionary, min_dictionary, database, means)
  err += temp_err
  # if err < best_err:
  #   best_means = means
  #   best_err = err
  #   best_cluster = clusters
  #   sillhoutte_dataset = []
  #   sillhoutte_label = []
  #   for cluster in best_cluster:
  #     for data in best_cluster[cluster]:
  #       sillhoutte_dataset.append(data)
  #       sillhoutte_label.append(cluster)

  #   sillhoutte_dataset = np.array(sillhoutte_dataset)

  #   pairwise_distance = calculate_pairwise_distance(sillhoutte_dataset, attribute_dictionary, max_dictionary, min_dictionary)

  #   # print(pairwise_distance)

  #   silhouette_avg = silhouette_score(pairwise_distance, sillhoutte_label, metric="precomputed")
  #   print("Sillhoutte score ->", silhouette_avg)
  #   print("Best error ->", best_err)

  print("Best error ->", err/iterations)

means [0, 1, 2]
[4, 1, 2]
[4, 14, 2]
[4, 14, 1]
means [4, 14, 1]
[24, 14, 1]
[24, 9, 1]
[24, 9, 5]
means [24, 9, 5]
[3, 9, 5]
[3, 6, 5]
[3, 6, 15]
means [3, 6, 15]
[18, 6, 15]
means [18, 6, 15]
Best error -> 0.5650564497744625
means [0, 1, 2]
[21, 1, 2]
[21, 17, 2]
[21, 17, 4]
means [21, 17, 4]
[5, 17, 4]
[5, 17, 0]
means [5, 17, 0]
[11, 17, 0]
means [11, 17, 0]
[13, 17, 0]
means [13, 17, 0]
Best error -> 1.1301128995489251
means [0, 1, 2]
[6, 1, 2]
[6, 7, 2]
[6, 7, 11]
means [6, 7, 11]
[22, 7, 11]
[22, 13, 11]
means [22, 13, 11]
[12, 13, 11]
means [12, 13, 11]
Best error -> 1.6951693493233877


In [ ]:
print(best_means)

[]


In [ ]:
for ind in best_cluster:
  print(ind, len(best_cluster[ind]))

In [ ]:
l = [0,1,2,'hi']
arr = np.array(l)
arr[0] = float(arr[0]) + 11
print(arr)

['11.0' '1' '2' 'hi']


In [ ]:
print(best_err)

9223372036854775807
